In [1]:
pip install mllib

  Preparing metadata (setup.py) ... done
  Created wheel for mllib: filename=mllib-1.0.0a2-py3-none-any.whl size=11906 sha256=1ae292e1f2ba435fb896e08afa74d3204f13612048ca6eb532d0bdd442525fff
  Stored in directory: /Users/clodette0071/Library/Caches/pip/wheels/e3/04/d8/1579c1f5cc5cc36409c9d803403153c3e959e7fc2fca6fa7b6
Successfully built mllib
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 7.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark import SparkConf, SparkContext
from numpy import array

In [2]:
#configuration du contexte
conf = SparkConf().setMaster("local").setAppName("SparkDecisionTree")
sc = SparkContext(conf = conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/20 21:35:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
#charger le fichier CSV et l'afficher 
rawData = sc.textFile("PastHires.csv")
rawData.take(10)

['Years Experience,Employed?,Previous employers,Level of Education,Top-tier school,Interned,Hired',
 '10,Y,4,BS,N,N,Y',
 '0,N,0,BS,Y,Y,Y',
 '7,N,6,BS,N,N,N',
 '2,Y,1,MS,Y,N,Y',
 '20,N,2,PhD,Y,N,N',
 '0,N,0,PhD,Y,Y,Y',
 '5,Y,2,MS,N,Y,Y',
 '3,N,1,BS,N,Y,Y',
 '15,Y,5,BS,N,N,Y']

In [7]:
#supprimer la première ligne (header) du fichier 
header = rawData.first()
rawData = rawData.filter(lambda x : x!=header)

In [3]:
# la fonction binary convertit chaque valeur booléenne (Y/N) en une donnée numérique
def binary(YN):
    if (YN == 'Y'):
        return 1
    else:
        return 0

# la fonction mapEducation convertit chaque valeur de la colonne 'diplome' de chaque candidat en une donnée numérique
def mapEducation(degree):
    if (degree == 'BS'):
        return 1
    elif (degree =='MS'):
        return 2
    elif (degree == 'PhD'):
        return 3
    else:
        return 0

In [8]:
#diviser chque ligne du RDD sur le delimiteur virgule
csvData = rawData.map(lambda x: x.split(","))

In [9]:
# Convertir chaque liste de données de notre CSV en
# LabeledPoint que MLLib peut utiliser. Toutes les données doivent être numériques, on utilise donc 
#les fonctions binary et mapEducation sur les colonnes appropriées. Pour les reste des champs, on s'assure que 
#ceux-ci soient numériques en les convertissant en int
def createLabeledPoints(fields):
    yearsExperience = int(fields[0])
    employed = binary(fields[1])
    previousEmployers = int(fields[2])
    educationLevel = mapEducation(fields[3])
    topTier = binary(fields[4])
    interned = binary(fields[5])
    hired = binary(fields[6])

    return LabeledPoint(hired, array([yearsExperience, employed,
        previousEmployers, educationLevel, topTier, interned]))

In [10]:
# convertir les lignes en LabeledPoints
trainingData = csvData.map(createLabeledPoints)

In [11]:
# Créer un candidat test, avec 10 ans d'expérience, actuellement employé,
# 3 employeurs précédents, un diplôme de licence, mais d'une école qui n'est pas de premier plan et 
#où il n'a pas fait de stage.
testCandidates = [ array([10, 1, 3, 1, 0, 0])]
testData = sc.parallelize(testCandidates)

In [12]:
# Entrainer l'arbre de classification avec nos données d'entrainement
model = DecisionTree.trainClassifier(trainingData, numClasses=2,
                                     categoricalFeaturesInfo={1:2, 3:4, 4:2, 5:2},
                                     impurity='gini', maxDepth=5, maxBins=32)

# faire la prédiction sur la donnée de test
predictions = model.predict(testData)
print('Hire prediction:')
results = predictions.collect()
for result in results:
    print(result)

# afficher l'arbre de décision lui même
print('Learned classification tree model:')
print(model.toDebugString())

23/06/20 22:31:44 WARN DecisionTreeMetadata: DecisionTree reducing maxBins from 32 to 13 (= number of training instances)


Hire prediction:
1.0
Learned classification tree model:
DecisionTreeModel classifier of depth 4 with 9 nodes
  If (feature 1 in {0.0})
   If (feature 5 in {0.0})
    If (feature 0 <= 0.5)
     If (feature 3 in {1.0})
      Predict: 0.0
     Else (feature 3 not in {1.0})
      Predict: 1.0
    Else (feature 0 > 0.5)
     Predict: 0.0
   Else (feature 5 not in {0.0})
    Predict: 1.0
  Else (feature 1 not in {0.0})
   Predict: 1.0

